In [1]:
import json
import os
from dotenv import load_dotenv
from termcolor import colored  

from openai import OpenAI

load_dotenv()

GPT_MODEL = "gpt-3.5-turbo-0613"
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [2]:
def chat_completion_request(messages, tools=None, tool_choice="auto", model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [3]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


# Single data extraction

You can easily extract structured data.

https://platform.openai.com/docs/guides/function-calling

In [4]:
tools = [{
        "type": "function",
        "function": {
            "name": "extract_people_information",
            "description": "Extract the information for a person. Make sure to use MM/DD/YY formatting for dates.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Name of the person to be extracted."
                    },
                    "birthday": {
                        "type": "string",
                        "description": "Date of birth of the person to be extracted."
                    },
                    "location": {
                        "type": "string",
                        "description": "Location of the person to be extracted."
                    }
                },
                "required": ["name, birthday, location"],
            },
        }
    }
]

You can create these definitions using pydantic as well.

In [21]:
from pydantic import BaseModel, Field

class extract_people_information(BaseModel):
    """Extract the information for a person. Make sure to use MM/DD/YY formatting for dates."""

    name: str = Field(description="Name of the person to be extracted.")
    birthday: str = Field(description="Date of birth of the person to be extracted.")
    location: str = Field(description="Location of the person to be extracted.")

print(json.dumps(extract_people_information.model_json_schema(), indent=2))

{
  "description": "Extract the information for a person. Make sure to use MM/DD/YY formatting for dates.",
  "properties": {
    "name": {
      "description": "Name of the person to be extracted.",
      "title": "Name",
      "type": "string"
    },
    "birthday": {
      "description": "Date of birth of the person to be extracted.",
      "title": "Birthday",
      "type": "string"
    },
    "location": {
      "description": "Location of the person to be extracted.",
      "title": "Location",
      "type": "string"
    }
  },
  "required": [
    "name",
    "birthday",
    "location"
  ],
  "title": "extract_people_information",
  "type": "object"
}


In [5]:
messages = []
messages.append({"role": "system", "content": "You're an assistant that summarizes the information for all people mentioned in the user messages."})
messages.append({"role": "user", "content": "I met Alessio, a guy from Rome whose birthday is on Aug 29th and is 29 years old (we are in 2024 now). I also met Sarah who was born on the 8th of August instead and is from Milwaukee"})
chat_response = chat_completion_request(messages, tools)
print(chat_response.choices)
assistant_message = chat_response.choices[0].message
assistant_message.content = str(assistant_message.tool_calls[0].function)
messages.append({"role": assistant_message.role, "content": assistant_message.content})

[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_zKSgfhxVKx26vdhr4ZNGBnR8', function=Function(arguments='{\n  "name": "Alessio",\n  "birthday": "08/29/1995",\n  "location": "Rome"\n}', name='extract_people_information'), type='function')]))]


In [6]:
pretty_print_conversation(messages)

print(assistant_message.tool_calls[0].function.arguments)

system: You're an assistant that summarizes the information for all people mentioned in the user messages.

user: I met Alessio, a guy from Rome whose birthday is on Aug 29th and is 29 years old (we are in 2024 now). I also met Sarah who was born on the 8th of August instead and is from Milwaukee

assistant: Function(arguments='{\n  "name": "Alessio",\n  "birthday": "08/29/1995",\n  "location": "Rome"\n}', name='extract_people_information')

{
  "name": "Alessio",
  "birthday": "08/29/1995",
  "location": "Rome"
}


No mention of Sarah in that message as you can see. One way to have multiple calls is defining the properties within an array:

In [7]:
tools = [{
    "type": "function",
    "function": {
        "name": "extract_people_information",
        "description": "Extract information for multiple people. Make sure to use MM/DD/YY formatting for dates.",
        "parameters": {
            "type": "object",
            "properties": {
                "people": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": {
                                "type": "string",
                                "description": "Name of the person to be extracted."
                            },
                            "birthday": {
                                "type": "string",
                                "description": "Date of birth of the person to be extracted."
                            },
                            "location": {
                                "type": "string",
                                "description": "Location of the person to be extracted."
                            }
                        },
                        "required": ["name", "birthday", "location"]
                    }
                }
            },
            "required": ["people"]
        },
    }
}]


In [8]:
messages = []
messages.append({"role": "system", "content": "You're an assistant that summarizes the information for all people mentioned in the user messages. You should make sure you don't miss anyone that is mentioned. There will be multiple people in each message."})
messages.append({"role": "user", "content": "I met Alessio, a guy from Rome whose birthday is on Aug 29th and is 29 years old (we are in 2024 now). I also met Sarah who was born on the 8th of August instead and is from Milwaukee"})
chat_response = chat_completion_request(messages, tools=tools, model="gpt-4-turbo-preview")
print(chat_response.choices)
assistant_message = chat_response.choices[0].message
assistant_message.content = str(assistant_message.tool_calls[0].function)
messages.append({"role": assistant_message.role, "content": assistant_message.content})

[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_f6w7w6IpdSOZvFJ1JNgng0i9', function=Function(arguments='{"people": [{"name": "Alessio", "birthday": "08/29/1995", "location": "Rome"}, {"name": "Sarah", "birthday": "08/08/??", "location": "Milwaukee"}]}', name='extract_people_information'), type='function')]))]


In [9]:
pretty_print_conversation(messages)

print(assistant_message.tool_calls[0].function.arguments)

system: You're an assistant that summarizes the information for all people mentioned in the user messages. You should make sure you don't miss anyone that is mentioned. There will be multiple people in each message.

user: I met Alessio, a guy from Rome whose birthday is on Aug 29th and is 29 years old (we are in 2024 now). I also met Sarah who was born on the 8th of August instead and is from Milwaukee

assistant: Function(arguments='{"people": [{"name": "Alessio", "birthday": "08/29/1995", "location": "Rome"}, {"name": "Sarah", "birthday": "08/08/??", "location": "Milwaukee"}]}', name='extract_people_information')

{"people": [{"name": "Alessio", "birthday": "08/29/1995", "location": "Rome"}, {"name": "Sarah", "birthday": "08/08/??", "location": "Milwaukee"}]}


Rather than defining functions, you can also use Instructor. By default, it's still a single call.

In [11]:
import instructor

from pydantic import BaseModel
client = instructor.patch(OpenAI())

class PersonDetail(BaseModel):
    name: str
    role: str
    company: str    

user = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=PersonDetail,
    messages=[
        {"role": "user", 
         "content": """
         Extract Alessio [00:00:00]: Hey everyone, welcome to the Latent Space podcast. This is Alessio, partner and CTO in Residence at Decibel Partners, and I'm joined by my co-host Swyx, founder of Smol AI.

                Swyx [00:00:14]: Hey, and today we have Ben Firshman in the studio. Welcome Ben.

                Ben [00:00:18]: Hey, good to be here.

                Swyx [00:00:19]: Ben, you're a co-founder and CEO of Replicate. Before that, you were most notably founder of Fig, which became Docker Compose. You also did a couple of other things before that, but that's what a lot of people know you for. What should people know about you that, you know, outside of your, your sort of LinkedIn profile?
         """},
    ],
)

assert isinstance(user, PersonDetail)
print(user.model_dump_json(indent=2))

{
  "name": "Ben Firshman",
  "role": "Co-founder and CEO",
  "company": "Replicate"
}


GPT-4-turbo-preview includes parallel function calling. Instructor still only has "early access" for it, but it works well:

In [14]:
import instructor

from typing import Iterable
from pydantic import BaseModel

client = instructor.patch(OpenAI(), mode=instructor.Mode.PARALLEL_TOOLS)  

class PersonDetail(BaseModel):
    name: str
    role: str
    company: str    

function_calls = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    response_model=Iterable[PersonDetail],
    messages=[
        {"role": "system", "content": "You must always use tools"},
        {"role": "user", 
         "content": """
         Extract Alessio [00:00:00]: Hey everyone, welcome to the Latent Space podcast. This is Alessio, partner and CTO in Residence at Decibel Partners, and I'm joined by my co-host Swyx, founder of Smol AI.

                Swyx [00:00:14]: Hey, and today we have Ben Firshman in the studio. Welcome Ben.

                Ben [00:00:18]: Hey, good to be here.

                Swyx [00:00:19]: Ben, you're a co-founder and CEO of Replicate. Before that, you were most notably founder of Fig, which became Docker Compose. You also did a couple of other things before that, but that's what a lot of people know you for. What should people know about you that, you know, outside of your, your sort of LinkedIn profile?
         """},
    ],
)

for fc in function_calls:
    print(fc)

name='Alessio' role='partner and CTO in Residence' company='Decibel Partners'
name='Swyx' role='founder' company='Smol AI'
name='Ben Firshman' role='co-founder and CEO' company='Replicate'


OpenAI's SDK also supports parallel function calling. We can re-use the exact same tools definition as before, but simply switch to GPT-4-turbo-preview and it works:

In [15]:
tools = [{
        "type": "function",
        "function": {
            "name": "extract_people_information",
            "description": "Extract the information for a person. Make sure to use MM/DD/YY formatting for dates.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Name of the person to be extracted."
                    },
                    "birthday": {
                        "type": "string",
                        "description": "Date of birth of the person to be extracted."
                    },
                    "location": {
                        "type": "string",
                        "description": "Location of the person to be extracted."
                    }
                },
                "required": ["name, birthday, location"],
            },
        }
    }
]

In [17]:
messages = []
messages.append({"role": "system", "content": "You're an assistant that summarizes the information for all people mentioned in the user messages."})
messages.append({"role": "user", "content": "I met Alessio, a guy from Rome whose birthday is on Aug 29th and is 29 years old (we are in 2024 now). I also met Sarah who was born on the 8th of August instead and is from Milwaukee"})
chat_response = chat_completion_request(messages, tools=tools, model='gpt-4-turbo-preview')
print(chat_response.choices)
assistant_message = chat_response.choices[0].message
assistant_message.content = str(assistant_message.tool_calls[0].function)
messages.append({"role": assistant_message.role, "content": assistant_message.content})

[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0rJeA4rdQXju9ZkyfJVFX7bM', function=Function(arguments='{"name": "Alessio", "birthday": "08/29/1995", "location": "Rome"}', name='extract_people_information'), type='function'), ChatCompletionMessageToolCall(id='call_l7XCtBxmnR54p9YwKl1vZcky', function=Function(arguments='{"name": "Sarah", "birthday": "08/08/1996", "location": "Milwaukee"}', name='extract_people_information'), type='function')]))]


In [20]:
pretty_print_conversation(messages)

for tool in assistant_message.tool_calls:
    print(tool.function.arguments)

system: You're an assistant that summarizes the information for all people mentioned in the user messages.

user: I met Alessio, a guy from Rome whose birthday is on Aug 29th and is 29 years old (we are in 2024 now). I also met Sarah who was born on the 8th of August instead and is from Milwaukee

assistant: Function(arguments='{"name": "Alessio", "birthday": "08/29/1995", "location": "Rome"}', name='extract_people_information')

{"name": "Alessio", "birthday": "08/29/1995", "location": "Rome"}
{"name": "Sarah", "birthday": "08/08/1996", "location": "Milwaukee"}
